In [1]:
from __future__ import print_function

import numpy as np
import tensorflow as tf

#this is the same util2 used for the first assigment
from util2 import Arff2Skl
import arff

from sklearn.model_selection import KFold
from sklearn.metrics import matthews_corrcoef
from time import time
import random

%load_ext autoreload
%autoreload 2

In [2]:
#parse the phishing.arff dataset
cvt = Arff2Skl("phishing.arff")
label = np.array(cvt.meta.names())
print(label)

['having_IP_Address' 'URL_Length' 'Shortining_Service' 'having_At_Symbol'
 'double_slash_redirecting' 'Prefix_Suffix' 'having_Sub_Domain'
 'SSLfinal_State' 'Domain_registeration_length' 'Favicon' 'port'
 'HTTPS_token' 'Request_URL' 'URL_of_Anchor' 'Links_in_tags' 'SFH'
 'Submitting_to_email' 'Abnormal_URL' 'Redirect' 'on_mouseover'
 'RightClick' 'popUpWidnow' 'Iframe' 'age_of_domain' 'DNSRecord'
 'web_traffic' 'Page_Rank' 'Google_Index' 'Links_pointing_to_page'
 'Statistical_report' 'Result']


In [3]:
#get the X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split

X, y = cvt.transform()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [4]:
#modifythe type from np.float64 (default type) to np.float32
X_train = X_train.astype(np.float32)
y_train = y_train.astype(np.int32)
X_test = X_test.astype(np.float32)
y_test = y_test.astype(np.int32)

In [6]:
dataset = arff.load(open('phishing.arff', 'r'))
data = np.array(dataset['data'])

In [7]:
print(data.shape)
#y = data[:,-1]
X = data[:,:-1]
print(y.shape)
print(X.shape)

#get the X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split

_, y = cvt.transform()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=33)

(11055, 31)
(11055, 2)
(11055, 30)


In [8]:
#modifythe type from np.float64 (default type) to np.float32
X_train = X_train.astype(np.float32)
y_train = y_train.astype(np.int32)
X_test = X_test.astype(np.float32)
y_test = y_test.astype(np.int32)
#print the shape of the matrix's
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(7406, 30)
(7406, 2)
(3649, 30)
(3649, 2)


## Parameters for the model

In [16]:
# Parameters for the classifier
learning_rate = 0.001
batch_size = 128 #the classifier will take blocks of 128 to run the classifier
display_step = 5
num_steps = int(X_train.shape[0]/batch_size) # 58 = ceil(7406/128) = ceil(57.85)


# Network Parameters
n_hidden_1 = 256 # 1st layer number of neurons
n_hidden_2 = 256 # 2nd layer number of neurons
num_input = 30 # number of features
num_classes = 2 # total amount of classes

# tf Graph input
X = tf.placeholder(tf.float32, [None], name="input")
Y = tf.placeholder(tf.int32, [None, num_classes], name="outputY")

## Create the model

this code create the model with random values for each weight and bias

In [17]:
# Store layers weight & bias (this should work without any change)
weights = {
    'h1': tf.Variable(tf.random_normal([num_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, num_classes]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([num_classes]))
}

In [18]:
# Create model (this should work without any change)
def neural_net(x):
    x_matrix = tf.reshape(x, [-1, num_input])
    # Hidden fully connected layer with 256 neurons
    layer_1 = tf.add(tf.matmul(x_matrix, weights['h1']), biases['b1'])
    # Hidden fully connected layer with 256 neurons
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    # Output fully connected layer with a neuron for each class
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer

In [19]:
# Construct model (this should work without any change)
logits = neural_net(X)
sofmax_out = tf.nn.softmax(logits)
# Define loss and optimizer
out = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y, name="output2")
loss_op = tf.reduce_mean(out)
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# Evaluate model (with test logits, for dropout to be disabled)
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

In [20]:
print(X_train[0:batch_size].shape)
#from Math import floor
print(int(7406/128))

(128, 30)
57


## Train the model

this code train the model and find the most optimal value for each weight and every bias

In [21]:
# Start training
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)

    for i in range(1, num_steps+1):
        batch_x = X_train[(i-1)*batch_size: (i-1)*batch_size + batch_size] #take a block of 128 instances
        batch_y = y_train[(i-1)*batch_size: (i-1)*batch_size + batch_size] #take a block of 128 instances
        batch_x = batch_x.ravel().astype(np.float32)
        #print(type(batch_x))
        #print(type(batch_y))
        
        # Run optimization op (backprop)
        sess.run(train_op, feed_dict={X: batch_x, Y: batch_y})
       

    print("Optimization Finished!")

    # Calculate accuracy for test instances
    print("Testing Accuracy:", sess.run(accuracy, feed_dict={X: X_test.ravel(), Y: y_test}))
    
    y_out = sess.run(sofmax_out, feed_dict={X: X_test.ravel()})
    
    #all_saver = tf.train.Saver()
    #all_saver.save(sess, '/model/model.ckpt')
    
    #this variables are used for the binary model, catch the current state for every weight and every bias
    W1 = weights['h1'].eval(sess)
    B1 = biases['b1'].eval(sess)
    W2 = weights['h2'].eval(sess)
    B2 = biases['b2'].eval(sess)
    W_OUT = weights['out'].eval(sess)
    B_OUT = biases['out'].eval(sess)
    
    #print(sess.run(weights.get('h1')))
    

Optimization Finished!
Testing Accuracy: 0.890929


In [22]:
print(y_out)
print(np.mean((y_test - y_out.round()) == 0))
print(matthews_corrcoef(y_test[:,0], y_out.round()[:,0]))

[[ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 ..., 
 [ 1.  0.]
 [ 0.  1.]
 [ 0.  1.]]
0.89092902165
0.779027089455


## Export the Model

This Code export the model as a binary file

In [96]:
# Create new graph for exporting

DIR = '/model/' # path where the model will be create it
g = tf.Graph()
with g.as_default():
    
    # this steps are the same as the function neural_net() but instead of random values will 
    # take the value in every weight and every bias at the end of the classifier
    
    #create the input parameter which is a vector
    x_2 = tf.placeholder(tf.float32, shape=[None], name="input")
    
    #reshape the vector into a matrix, this has to be done because the the op matmul
    #requires two matrix's
    x_22 = tf.reshape(x_2, [-1, num_input])
    
    #create each weight and each biase with the final value
    W1_C = tf.constant(W1, name="W1")
    B1_C = tf.constant(B1, name="B1")
    W2_C = tf.constant(W2, name="W2")
    B2_C = tf.constant(B2, name="B2")
    W_OUT_C = tf.constant(W_OUT, name="W_OUT")
    B_OUT_C = tf.constant(B_OUT, name="B_OUT")
    
    
    # the two layers as used in neural_net()
    layer_1 = tf.add(tf.matmul(x_22, W1_C, name="matmul_x_22_w1_c"), B1_C, name="add_matmul1_b1_c")
    layer_2 = tf.add(tf.matmul(layer_1, W2_C, name="matmul_l_1_w1_c"), B2_C, name="add_matmul2_b2_c")
    #the output value of the classifier with the name output
    OUTPUT = tf.nn.softmax(tf.matmul(layer_2, W_OUT, name="matmul_l_2_wout_c") + B_OUT, name="output")
    

    # skipped dropout for exported graph as there is no need for already calculated weights
    sess = tf.Session()
    init = tf.global_variables_initializer()
    sess.run(init)

    graph_def = g.as_graph_def()
    
    #create the model model_graph.pb
    tf.train.write_graph(graph_def, DIR, 'phishing_model_opt4.pb', as_text=False)

    
    # Test trained model
    y_train = tf.placeholder(tf.float32, [None, num_classes])
    correct_prediction = tf.equal(tf.argmax(OUTPUT, 1), tf.argmax(y_train, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    print("check accuracy %g" % accuracy.eval({x_2: X_test.ravel(), y_train: y_test}, sess))

check accuracy 0.904905


In [39]:
print(X_test[1])
print(y_test[1])

[-1. -1.  1. -1.  1. -1.  1. -1.  1. -1. -1.  1. -1. -1.  0.  1. -1. -1.
  1.  0.  1.  1.]
[1 0]


In [116]:
print(y_test.shape)

(339, 3)


## Use of the custom library made

In [28]:
%reload_ext autoreload

In [11]:
from neuron import *
from sklearn.model_selection import KFold
from sklearn.metrics import matthews_corrcoef
from time import time
import random

In [9]:
#get the samples
dataset = arff.load(open('phishing2.arff', 'r'))
data = np.array(dataset['data'])
X = data[:,:-1]

#get the labels
cvt = Arff2Skl("phishing2.arff")
_, y = cvt.transform()


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=10)


In [12]:
accs = np.zeros(5*10).reshape(5,10)
mccs = np.zeros(5*10).reshape(5,10)
t1 = time()
for j ,i in enumerate(range(219, 224)):
    ##get a number random set of train and test
    kf = KFold(n_splits=10, shuffle=True, random_state=random.randint(1,100))
    k = 0
    for train, test in kf.split(X):
        X_train, X_test, y_train, y_test = X[train], X[test], y[train], y[test]
        clf = NeuronNetworkTwoHiddenLayer(n_hidden_1 = 256+i, n_hidden_2 = 256+i)
        clf.train(X_train, y_train)
        y_out = clf.predict(X_test)
        accs[j][k] = matthews_corrcoef(y_test[:,0], y_out.round()[:,0])
        mccs[j][k] = np.mean((y_test - y_out.round()) == 0)
        k +=1
        clf.clear()
        del y_out
        del clf
    del kf
    print(j)
print('Done in ', time() - t1)

0
1
2
3
4
Done in  541.8984749317169
